In [ ]:
from huggingface_hub import login

login("")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor
from PIL import Image

# Initialize the model and processor
model_id = "samihalawa/APOLO-medical-multimodal-instruct"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
model = model.to(torch.bfloat16).cuda().eval()



SyntaxError: unterminated string literal (detected at line 6) (2950224540.py, line 6)

In [ ]:

# Prepare a clinical query with an image
conversation = [
    {
        "role": "User",
        "content": "<image>\nDescribe the key findings in this chest X-ray.",
        "images": ["path/to/anonymized_chest_xray.jpg"],
    },
    {"role": "Assistant", "content": ""}
]

# Process images and prepare inputs
images = [Image.open(img_path) for img_path in conversation[0]["images"]]
inputs = processor(
    conversations=conversation,
    images=images,
    force_batchify=True,
    privacy_mode=True  # Ensures Stage 1 -> Stage 2 privacy enforcement
).to(model.device)

# Generate response with reasoning
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True
    )

response = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(response)